In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


In [3]:

import os
if not os.path.isdir('saved_models_{}'.format('sngan')):
    os.mkdir('saved_models_{}'.format('sngan'))
f = open('saved_models_{}/log_collapse1.txt'.format('sngan'), mode='w')
import os
import tensorflow as tf
import torch.utils.data as Data
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10,fashion_mnist,mnist
import os
from scipy import misc
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose, Conv2D
from keras.optimizers import Adam
import os
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
import cv2
import keras.backend.tensorflow_backend as KTF

from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(1,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(SNConv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(0., 1 - pred_real)) + K.mean(K.maximum(0., 1 + pred_fake))
    g_loss = -K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train

class SNConv2D(_Conv):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(SNConv2D, self).__init__(
            rank=2,
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

        self.input_spec = InputSpec(ndim=4)
        self.Ip = 1
        self.u = self.add_weight(
            name='W_u',
            shape=(1,filters),
            initializer='random_uniform',
            trainable=False
        )

    def call(self, inputs):
        outputs = K.conv2d(
            inputs,
            self.W_bar(),
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs


    def get_config(self):
        config = super(SNConv2D, self).get_config()
        config.pop('rank')
        return config

    def W_bar(self):
        # Spectrally Normalized Weight
        W_mat = K.permute_dimensions(self.kernel, (3, 2, 0, 1)) # (h, w, i, o) => (o, i, h, w)
        W_mat = K.reshape(W_mat,[K.shape(W_mat)[0], -1]) # (o, i * h * w)

        if not self.Ip >= 1:
            raise ValueError("The number of power iterations should be positive integer")

        _u = self.u
        _v = None

        for _ in range(self.Ip):
            _v = _l2normalize(K.dot(_u, W_mat))
            _u = _l2normalize(K.dot(_v, K.transpose(W_mat)))

        sigma = K.sum(K.dot(_u,W_mat)*_v)

        K.update(self.u,K.in_train_phase(_u, self.u))
        return self.kernel / sigma

def _l2normalize(x):
    return x / K.sqrt(K.sum(K.square(x)) + K.epsilon())
set_gpu_config("0",0.5)

epochs = 50
image_size = (32,32,1)
noise_size = (2,2,32)
batch_size = 64
sample_size=10
size=32
sample_interval=200
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=[cv2.resize(i,(32,32)) for i in x_train]

x_train=np.array(x_train)
x_test=np.array(x_test)
x_train=np.expand_dims(x_train,axis=3)
x_test=np.expand_dims(x_test,axis=3)
num_of_data = x_train.shape[0]
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train/255)*2-1
x_test = (x_test/255)*2-1
y_train = keras.utils.to_categorical(y_train,10)
# y_test = keras.utils.to_categorical(y_test,10)
x = []
y = np.zeros((31, 1), dtype=np.int)
y = list(y)
for i in range(31):
    y[i] = []
generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

nb_batches = int(x_train.shape[0] / batch_size)
global_step = 0
for epoch in range(epochs):
    for index in range(nb_batches):
        global_step += 1
        real_images = x_train[index * batch_size:(index + 1) * batch_size]
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
        sampleSize=10000
        if global_step % sample_interval == 0:
            image = generator.predict(np.random.normal(size=(sampleSize,) + noise_size))
            image = 255 * (image + 1) / 2
            image = [cv2.resize(i, (28, 28)) for i in image]
            image = np.expand_dims(image, axis=3)
            image = np.array(image)
            labels = np.squeeze(y_test[:sampleSize])
            labels = np.squeeze(labels)
            dis_real = cal_distance_image_real(x_test[:sampleSize], labels)
            dis_fake = cal_distance_image_fake(image )
            dis_cha = dis_real - dis_fake
            print('##############')
            # print(dis_real)
            # print(dis_fake)
            print(dis_cha)
            print('##########')
            f.writelines('\n')
            f.writelines('epoch:' + str(epoch))
            f.writelines('\n')
            f.writelines('紧度')
            f.writelines('\n')
            f.writelines(' %.8f ' % (i) for i in dis_cha)
            f.writelines('\n')



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_5 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 32, 32, 64)        73792     
__________

[0/50] [0_123] d_loss: 0.02106, g_loss: 1.622
[0/50] [0_124] d_loss: 0.02488, g_loss: 1.455
[0/50] [0_125] d_loss: 0.04454, g_loss: 1.63
[0/50] [0_126] d_loss: 0.05848, g_loss: 1.49
[0/50] [0_127] d_loss: 0.07181, g_loss: 1.648
[0/50] [0_128] d_loss: 0.1121, g_loss: 1.177
[0/50] [0_129] d_loss: 0.1314, g_loss: 2.516
[0/50] [0_130] d_loss: 0.1085, g_loss: 1.763
[0/50] [0_131] d_loss: 0.04861, g_loss: 1.819
[0/50] [0_132] d_loss: 0.006421, g_loss: 1.733
[0/50] [0_133] d_loss: 0.01123, g_loss: 1.876
[0/50] [0_134] d_loss: 0.03532, g_loss: 1.69
[0/50] [0_135] d_loss: 0.05868, g_loss: 2.813
[0/50] [0_136] d_loss: 0.2836, g_loss: 1.144
[0/50] [0_137] d_loss: 1.446, g_loss: 2.72
[0/50] [0_138] d_loss: 2.138, g_loss: 0.3244
[0/50] [0_139] d_loss: 0.6925, g_loss: 0.3777
[0/50] [0_140] d_loss: 0.6442, g_loss: 0.5043
[0/50] [0_141] d_loss: 0.5351, g_loss: 0.7366
[0/50] [0_142] d_loss: 0.2788, g_loss: 1.323
[0/50] [0_143] d_loss: 0.0209, g_loss: 1.3
[0/50] [0_144] d_loss: 0.03203, g_loss: 1.528
[0

[0/50] [0_308] d_loss: 0.4225, g_loss: 1.883
[0/50] [0_309] d_loss: 0.2453, g_loss: 1.459
[0/50] [0_310] d_loss: 0.3208, g_loss: 3.031
[0/50] [0_311] d_loss: 0.5493, g_loss: 0.3896
[0/50] [0_312] d_loss: 1.407, g_loss: 3.761
[0/50] [0_313] d_loss: 1.417, g_loss: 0.723
[0/50] [0_314] d_loss: 0.5816, g_loss: 2.685
[0/50] [0_315] d_loss: 0.2393, g_loss: 1.805
[0/50] [0_316] d_loss: 0.1852, g_loss: 1.787
[0/50] [0_317] d_loss: 0.3108, g_loss: 2.633
[0/50] [0_318] d_loss: 0.3835, g_loss: 1.194
[0/50] [0_319] d_loss: 0.8583, g_loss: 3.631
[0/50] [0_320] d_loss: 0.7201, g_loss: 0.7637
[0/50] [0_321] d_loss: 0.9274, g_loss: 2.846
[0/50] [0_322] d_loss: 0.8283, g_loss: 0.1881
[0/50] [0_323] d_loss: 1.435, g_loss: 2.343
[0/50] [0_324] d_loss: 1.021, g_loss: 0.344
[0/50] [0_325] d_loss: 1.0, g_loss: 1.616
[0/50] [0_326] d_loss: 0.6948, g_loss: 1.159
[0/50] [0_327] d_loss: 0.7101, g_loss: 2.379
[0/50] [0_328] d_loss: 0.8079, g_loss: 0.2161
[0/50] [0_329] d_loss: 1.27, g_loss: 2.963
[0/50] [0_330] 

[0/50] [0_491] d_loss: 0.9724, g_loss: 0.4113
[0/50] [0_492] d_loss: 0.8546, g_loss: 1.701
[0/50] [0_493] d_loss: 0.7469, g_loss: 0.8712
[0/50] [0_494] d_loss: 0.7526, g_loss: 1.536
[0/50] [0_495] d_loss: 0.8846, g_loss: 1.059
[0/50] [0_496] d_loss: 1.13, g_loss: 2.581
[0/50] [0_497] d_loss: 1.405, g_loss: -0.1733
[0/50] [0_498] d_loss: 1.59, g_loss: 0.9348
[0/50] [0_499] d_loss: 1.205, g_loss: 0.8724
[0/50] [0_500] d_loss: 0.9033, g_loss: 0.907
[0/50] [0_501] d_loss: 1.165, g_loss: 2.393
[0/50] [0_502] d_loss: 1.039, g_loss: -0.05989
[0/50] [0_503] d_loss: 1.772, g_loss: 1.535
[0/50] [0_504] d_loss: 0.9047, g_loss: 0.6863
[0/50] [0_505] d_loss: 0.941, g_loss: 1.315
[0/50] [0_506] d_loss: 0.8729, g_loss: 0.1434
[0/50] [0_507] d_loss: 0.9853, g_loss: 2.559
[0/50] [0_508] d_loss: 1.148, g_loss: 0.2233
[0/50] [0_509] d_loss: 1.37, g_loss: 1.356
[0/50] [0_510] d_loss: 0.6846, g_loss: 0.838
[0/50] [0_511] d_loss: 0.8662, g_loss: 1.334
[0/50] [0_512] d_loss: 0.6462, g_loss: 0.7136
[0/50] [0_

[0/50] [0_673] d_loss: 1.01, g_loss: 0.6853
[0/50] [0_674] d_loss: 0.8918, g_loss: 1.528
[0/50] [0_675] d_loss: 0.7903, g_loss: 0.4708
[0/50] [0_676] d_loss: 0.9312, g_loss: 0.8023
[0/50] [0_677] d_loss: 0.6396, g_loss: 1.147
[0/50] [0_678] d_loss: 0.6934, g_loss: 0.9536
[0/50] [0_679] d_loss: 0.6264, g_loss: 0.9467
[0/50] [0_680] d_loss: 0.7263, g_loss: 1.566
[0/50] [0_681] d_loss: 0.5889, g_loss: 1.835
[0/50] [0_682] d_loss: 0.6808, g_loss: 0.02612
[0/50] [0_683] d_loss: 1.396, g_loss: 3.448
[0/50] [0_684] d_loss: 1.413, g_loss: 0.2635
[0/50] [0_685] d_loss: 1.247, g_loss: 0.8241
[0/50] [0_686] d_loss: 0.9184, g_loss: 1.611
[0/50] [0_687] d_loss: 0.837, g_loss: 0.7875
[0/50] [0_688] d_loss: 1.198, g_loss: 2.341
[0/50] [0_689] d_loss: 1.377, g_loss: -0.5779
[0/50] [0_690] d_loss: 2.218, g_loss: 0.8895
[0/50] [0_691] d_loss: 0.997, g_loss: 0.8301
[0/50] [0_692] d_loss: 1.038, g_loss: 1.182
[0/50] [0_693] d_loss: 0.999, g_loss: 1.264
[0/50] [0_694] d_loss: 0.7522, g_loss: 1.909
[0/50] [

...

[49/50] [49_46151] d_loss: 0.01556, g_loss: 3.801
[49/50] [49_46152] d_loss: 0.0, g_loss: 2.814
[49/50] [49_46153] d_loss: 0.05665, g_loss: 2.414
[49/50] [49_46154] d_loss: 0.07099, g_loss: 3.979
[49/50] [49_46155] d_loss: 0.05689, g_loss: 3.27
[49/50] [49_46156] d_loss: 0.0, g_loss: 2.957
[49/50] [49_46157] d_loss: 0.03707, g_loss: 2.75
[49/50] [49_46158] d_loss: 0.002199, g_loss: 3.583
[49/50] [49_46159] d_loss: 0.1264, g_loss: 2.829
[49/50] [49_46160] d_loss: 0.0751, g_loss: 2.391
[49/50] [49_46161] d_loss: 0.008085, g_loss: 2.489
[49/50] [49_46162] d_loss: 0.0212, g_loss: 2.736
[49/50] [49_46163] d_loss: 0.08407, g_loss: 2.621
[49/50] [49_46164] d_loss: 0.09061, g_loss: 2.626
[49/50] [49_46165] d_loss: 0.009369, g_loss: 2.97
[49/50] [49_46166] d_loss: 0.003901, g_loss: 3.203
[49/50] [49_46167] d_loss: 0.1025, g_loss: 2.369
[49/50] [49_46168] d_loss: 0.01107, g_loss: 2.312
[49/50] [49_46169] d_loss: 0.01708, g_loss: 2.862
[49/50] [49_46170] d_loss: 0.002503, g_loss: 3.235
[49/50] [4

[49/50] [49_46320] d_loss: 0.0, g_loss: 2.358
[49/50] [49_46321] d_loss: 0.04927, g_loss: 1.965
[49/50] [49_46322] d_loss: 0.09507, g_loss: 1.883
[49/50] [49_46323] d_loss: 0.0287, g_loss: 4.069
[49/50] [49_46324] d_loss: 0.08162, g_loss: 1.933
[49/50] [49_46325] d_loss: 0.09943, g_loss: 1.544
[49/50] [49_46326] d_loss: 0.06924, g_loss: 2.423
[49/50] [49_46327] d_loss: 0.02679, g_loss: 2.39
[49/50] [49_46328] d_loss: 0.0062, g_loss: 2.359
[49/50] [49_46329] d_loss: 0.09327, g_loss: 2.139
[49/50] [49_46330] d_loss: 0.0, g_loss: 1.874
[49/50] [49_46331] d_loss: 0.02149, g_loss: 1.925
[49/50] [49_46332] d_loss: 0.06902, g_loss: 1.573
[49/50] [49_46333] d_loss: 0.02301, g_loss: 2.51
[49/50] [49_46334] d_loss: 0.04987, g_loss: 2.863
[49/50] [49_46335] d_loss: 0.05729, g_loss: 2.721
[49/50] [49_46336] d_loss: 0.02712, g_loss: 2.719
[49/50] [49_46337] d_loss: 0.05995, g_loss: 1.767
[49/50] [49_46338] d_loss: 0.01275, g_loss: 2.837
[49/50] [49_46339] d_loss: 0.06874, g_loss: 2.792
[49/50] [49_

[49/50] [49_46492] d_loss: 0.08932, g_loss: 2.033
[49/50] [49_46493] d_loss: 0.04242, g_loss: 3.172
[49/50] [49_46494] d_loss: 0.04261, g_loss: 2.099
[49/50] [49_46495] d_loss: 0.0522, g_loss: 1.705
[49/50] [49_46496] d_loss: 0.02502, g_loss: 1.402
[49/50] [49_46497] d_loss: 0.03964, g_loss: 2.265
[49/50] [49_46498] d_loss: 0.0, g_loss: 2.329
[49/50] [49_46499] d_loss: 0.0, g_loss: 2.368
[49/50] [49_46500] d_loss: 0.1119, g_loss: 2.305
[49/50] [49_46501] d_loss: 0.03921, g_loss: 1.699
[49/50] [49_46502] d_loss: 0.08584, g_loss: 3.541
[49/50] [49_46503] d_loss: 0.09576, g_loss: 2.868
[49/50] [49_46504] d_loss: 0.03136, g_loss: 2.599
[49/50] [49_46505] d_loss: 0.01988, g_loss: 2.301
[49/50] [49_46506] d_loss: 0.06026, g_loss: 3.408
[49/50] [49_46507] d_loss: 0.02608, g_loss: 3.056
[49/50] [49_46508] d_loss: 0.009931, g_loss: 4.16
[49/50] [49_46509] d_loss: 0.01316, g_loss: 4.277
[49/50] [49_46510] d_loss: 0.04288, g_loss: 2.99
[49/50] [49_46511] d_loss: 0.01304, g_loss: 2.891
[49/50] [49

[49/50] [49_46662] d_loss: 0.01215, g_loss: 2.608
[49/50] [49_46663] d_loss: 0.1013, g_loss: 1.579
[49/50] [49_46664] d_loss: 0.09593, g_loss: 1.907
[49/50] [49_46665] d_loss: 0.09032, g_loss: 1.454
[49/50] [49_46666] d_loss: 0.0638, g_loss: 1.813
[49/50] [49_46667] d_loss: 0.01299, g_loss: 2.453
[49/50] [49_46668] d_loss: 0.03651, g_loss: 2.673
[49/50] [49_46669] d_loss: 0.0, g_loss: 2.88
[49/50] [49_46670] d_loss: 0.076, g_loss: 2.04
[49/50] [49_46671] d_loss: 0.05148, g_loss: 1.99
[49/50] [49_46672] d_loss: 0.02429, g_loss: 3.29
[49/50] [49_46673] d_loss: 0.0, g_loss: 2.511
[49/50] [49_46674] d_loss: 0.01052, g_loss: 3.127
[49/50] [49_46675] d_loss: 0.007151, g_loss: 4.136
[49/50] [49_46676] d_loss: 0.001266, g_loss: 3.345
[49/50] [49_46677] d_loss: 0.02192, g_loss: 3.583
[49/50] [49_46678] d_loss: 0.01577, g_loss: 2.984
[49/50] [49_46679] d_loss: 0.02067, g_loss: 2.423
[49/50] [49_46680] d_loss: 0.1444, g_loss: 3.13
[49/50] [49_46681] d_loss: 0.005853, g_loss: 2.117
[49/50] [49_466

[49/50] [49_46830] d_loss: 0.04459, g_loss: 5.313
[49/50] [49_46831] d_loss: 0.009629, g_loss: 3.01
[49/50] [49_46832] d_loss: 0.01771, g_loss: 2.638
[49/50] [49_46833] d_loss: 0.06633, g_loss: 1.819
[49/50] [49_46834] d_loss: 0.02065, g_loss: 2.424
[49/50] [49_46835] d_loss: 0.02987, g_loss: 2.003
[49/50] [49_46836] d_loss: 0.04838, g_loss: 2.435
[49/50] [49_46837] d_loss: 0.0, g_loss: 2.547
[49/50] [49_46838] d_loss: 0.0, g_loss: 2.76
[49/50] [49_46839] d_loss: 0.0, g_loss: 2.608
[49/50] [49_46840] d_loss: 0.001411, g_loss: 3.016
[49/50] [49_46841] d_loss: 0.006281, g_loss: 3.182
[49/50] [49_46842] d_loss: 0.00462, g_loss: 2.964
[49/50] [49_46843] d_loss: 0.0, g_loss: 2.927
[49/50] [49_46844] d_loss: 0.01039, g_loss: 2.85
[49/50] [49_46845] d_loss: 0.02943, g_loss: 2.666
[49/50] [49_46846] d_loss: 0.0, g_loss: 2.468
[49/50] [49_46847] d_loss: 0.01225, g_loss: 3.076
[49/50] [49_46848] d_loss: 0.0, g_loss: 3.284
[49/50] [49_46849] d_loss: 0.01762, g_loss: 2.944
[49/50] [49_46850] d_los